In [1]:
import joblib
from flask import Flask, request, jsonify
import threading
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Step 1: Train a simple model (Logistic Regression on the Iris dataset)
# Load dataset
data = load_iris()
X = data.data
y = data.target

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
print(f"Model Accuracy: {accuracy_score(y_test, y_pred)}")

# Save the trained model
joblib.dump(model, 'model.pkl')

# Step 2: Create the Flask app
app = Flask(__name__)

@app.route('/')
def home():
    return "Welcome to the Machine Learning API! Use the /predict endpoint to make predictions."

@app.route('/favicon.ico')
def favicon():
    return '', 204  # Handle favicon.ico requests to avoid 404 errors

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()  # Get the JSON data sent in the POST request
    features = [data['features']]  # Extract the features for prediction
    prediction = model.predict(features)  # Make the prediction using the model
    return jsonify({'prediction': prediction.tolist()})  # Return the prediction in JSON format

# Function to run Flask in a separate thread (necessary for Jupyter notebook compatibility)
def run_flask():
    app.run(debug=True, use_reloader=False, host='0.0.0.0', port=5000)

# Start Flask app in a separate thread to avoid blocking Jupyter notebook
thread = threading.Thread(target=run_flask)
thread.start()

print("Flask app is running!")


Model Accuracy: 1.0
Flask app is running!
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.45.117:5000
Press CTRL+C to quit


In [2]:
import requests

# Define input data (for Iris dataset)
input_data = {
    "features": [5.1, 3.5, 1.4, 0.2]  # Example input for prediction
}

# Send POST request to Flask API
response = requests.post("http://127.0.0.1:5000/predict", json=input_data)

# Print the prediction result
print(response.json())


{'prediction': [0]}
